# Iterative Scaling

In [28]:
import pods
import mlai
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


### Preliminaries

**Auxiliary function**

An auxiliary function is a (pointwise) lower or upper bound on a function.
<img src="./figs/auxiliary.jpg", width=300, align=center>

Because $x - 1 \geq \log x$ (see the figure above), $x - 1$ is an auxiliary function for $\log x$ (and $\log x$ is an auxiliary function for $x - 1$).

**Maximum likelihood model**

Supposing that the symbol $y$ is observed $\tilde{c}(y)$ times in a sample sequence of size $T$, the probability of symbol $y$ is given by $\displaystyle \tilde{p}(y) = \frac{\tilde{c}(y)}{T}$.

The **likelihood** of the sample with respect to a model $p(y)$ is
$$
    {\cal P}(\tilde{p} | p) = \prod_{y\in {\cal Y}} p(y)^{\tilde{c}(y)}
$$

There is a monotone relation to the *average per symbol log likelihood*:
$$
    {\cal L}(\tilde{p} | p) = \sum_{y\in {\cal Y}} \tilde{p}(y) \log p(y)
$$

And we define the **maximum likelihood model** using the log likelohood:
$$
	p* = \mathop{\rm argmax}_p {\cal L}(\tilde{p} | p)
$$

We are interested in the gain in log likelihood, ${\cal L}(\tilde{p} | p_{\theta'}) - {\cal L}(\tilde{p} | p_{\theta})$, of using one model $p_{\theta'}$ instead of another model $p_{\theta}$ .

We define an *auxiliary function* ${\cal A}(\theta' | \theta)$ such that
\begin{align*}
	{\cal L}(\tilde{p} | p_{\theta'}) - {\cal L}(\tilde{p} | p_{\theta}) &\geq {\cal A}(\theta' | \theta) \\
	{\cal A}(\theta | \theta) &= 0
\end{align*}
    
Suppose we have a $\theta$ and, if we find a $\theta'$ that satisfies ${\cal A}(\theta' | \theta) > 0$, then $p_{\theta'}$ is a better model than $p_{\theta}$ (in a maximum likelihood sense).

Let $\displaystyle p_{\theta}(y) = \sum_{z\in {\cal Z}} p_{\theta}(y, z)$, and we have
\begin{align*}
    {\cal L}(\tilde{p} | p_{\theta'}) - {\cal L}(\tilde{p} | p_{\theta})
	&= \sum_{y\in {\cal Y}} \tilde{p}(y) \log \frac{p_{\theta'}(y)}{p_{\theta}(y)} \\
	&= \sum_{y\in {\cal Y}} \tilde{p}(y) \log \sum_{z\in {\cal Z}} \frac{p_{\theta'}(y, z) p_{\theta}(z | y)} {p_{\theta}(y, z)} \\
	&\geq \underbrace{\sum_{y\in {\cal Y}} \tilde{p}(y) \sum_{z\in {\cal Z}} p_{\theta}(z | y) \log \frac{p_{\theta'}(y, z)} {p_{\theta}(y, z)}}_{{\cal A}(\theta' | \theta)}
\end{align*}

**EM algorithm** starts with some initial value $\theta^{[0]}$, then iterates
- *E step*: compute ${\cal A}(\theta' | \theta)$
- *M step*: update $\displaystyle \theta \leftarrow \mathop{\rm argmax}_{\theta'}{\cal A}(\theta' | \theta)$

**Linear interpolation example**

We have observed samples $y = \{y_1, \ldots, y_T\}$ and a collection of models $p_1(y), \ldots, p_N(y)$.
We wish to find the maximum likelihood distribution of
$$
    p(y) = \sum_{i = 1\ldots N} c_i p_i(y)
$$
given a constraint $\displaystyle \sum_{i = 1\ldots N} c_i = 1$ .
This is an $(N - 1)$ dimensional search problem, which is easy by the **EM algorithm**.

Here's how the dataset looks like:
<img src="./figs/dataset.jpg", width=500, align=center>

We denote the probability of using the $i^{th}$ model $p_i(y_t)$ and producing output $y_t$ by
$$
    p_c(y_t, i) = c_i p_i(y_t)
$$
and the probability of using $p_i(y_t)$, suppose the current output is $y_t$, by
$$
    p_c(i | y_t) = \frac{c_i p_i(y_t)} {\displaystyle \sum_{j = 1\ldots N} c_j p_j(y_t)}
$$

Now the *auxiliary function* for $c = \{c_1, \ldots, c_N\}$ is written as
$$
    {\cal A}(c' | c) = \sum_{t = 1\ldots T} \tilde{p}(y_t) \sum_{i = 1\ldots N} p_c(i | y_t) \log\frac{p_{c'}(y_t, i)}{p_c(y_t, i)}
$$

Using a *Lagrange multiplier* $\gamma$,
$$
    0
    = \frac{\partial}{\partial c_i'} \left\{ {\cal A}(c' | c) + \gamma \left( \sum_{i = 1\ldots N} c_i' - 1 \right) \right\}
    = \frac{1}{c_i'} \sum_{t = 1\ldots T} \tilde{p}(y_t) p_c(i | y_t) + \gamma
$$
which results in $\gamma = -1$, and hence we update $c_i$ using
$$
    c_i'
    = \sum_{t = 1\ldots T} \tilde{p}(y_t) p_c(i | y_t)
    = \sum_{t = 1\ldots T} \tilde{p}(y_t) \frac{c_i p_i(y_t)} {\displaystyle \sum_{j = 1\ldots N} c_j p_j(y_t)}
$$

Here's a mixture of two Normal distributions:
<img src="./figs/mixture.jpg", width=500, align=center>

### Numerical Solution

We can formulate the task (ie, that of finding a solution to the maximum entropy model) as a pure *maximum likelihood* problem of discovering the optimal values for the model parameters.

We start with the log likelihood of the general solution $p_{\lambda}(y|x)$:
\begin{align*}
    {\cal L}(\tilde{p} | p_{\lambda}) &= \sum_{x\in {\cal X}} \sum_{y\in {\cal Y}} \tilde{p}(x,y) \log p_{\lambda}(y|x) \\
    &= \sum_{x\in {\cal X}} \sum_{y\in {\cal Y}} \tilde{p}(x,y) \sum_i \lambda_i f_i(x,y) - \sum_{x\in {\cal X}} \tilde{p}(x) \log Z_{\lambda}(x) \underbrace{\sum_{y\in {\cal Y}} \tilde{p}(y|x)}_1
\end{align*}

Differentiating ${\cal L}(\tilde{p} | p_{\lambda})$ with respect to an individual $\lambda_i$,
\begin{align*}
    \frac{\partial {\cal L}(\tilde{p} | p_{\lambda})}{\partial \lambda_i}
    &= \sum_{x\in {\cal X}} \sum_{y\in {\cal Y}} \tilde{p}(x,y) \frac{\partial}{\partial \lambda_i} \sum_i \lambda_i f_i(x,y) - \sum_{x\in {\cal X}} \tilde{p}(x) \frac{\partial \log Z_{\lambda}(x)}{\partial \lambda_i} \\
    &= \underbrace{\sum_{x\in {\cal X}} \sum_{y\in {\cal Y}} \tilde{p}(x,y) f_i(x,y)}_{\tilde{p}(f_i)} - \underbrace{\sum_{x\in {\cal X}} \tilde{p}(x) \sum_{y\in {\cal Y}} p_{\lambda}(y|x) f_i(x,y)}_{p(f_i)}
\end{align*}

Setting $\displaystyle \frac{\partial {\cal L}(\tilde{p} | p_{\lambda})}{\partial \lambda_i}$ to zero yields the condition for maximising the log likelihood with respect to $\lambda_i$.
In other words, the maximum entropy model $p*\in{\cal C}$ also maximises the *likelihood* of the training sample $\tilde{p}$ .

A **numerical solution** is the procedure that iteratively achieves
$$
    \lambda \equiv \{ \lambda_1, \ldots, \lambda_n \}
	\quad\longrightarrow\quad \lambda + \delta \equiv
	\{ \lambda_1 + \delta_1, \ldots, \lambda_n + \delta_n \}
$$
where $\lambda + \delta$ is not inferior to $\lambda$ in a *log likelihood* sense.

To achieve this, we calculate
\begin{align*}
    {\cal L}(\tilde{p} | p_{\lambda + \delta}) - {\cal L}(\tilde{p} | p_{\lambda})
    &= \sum_{x\in {\cal X}} \sum_{y\in {\cal Y}} \tilde{p}(x,y) \log p_{\lambda + \delta}(y|x) - \sum_{x\in {\cal X}} \sum_{y\in {\cal Y}} \tilde{p}(x,y) \log p_{\lambda}(y|x) \\
    &= \sum_{x\in {\cal X}} \sum_{y\in {\cal Y}} \tilde{p}(x,y) \sum_i \delta_i f_i(x,y) - \sum_{x\in {\cal X}} \tilde{p}(x) \log \frac{Z_{\lambda + \delta}(x)}{Z_{\lambda}(x)}
\end{align*}

Using the inequality $-\log a \geq 1 - a$ for all $a > 0$, we establish a lower bound on the change in the log likelihood:
$$
    {\cal L}(\tilde{p} | p_{\lambda + \delta}) - {\cal L}(\tilde{p} | p_{\lambda}) \geq {\cal A}(\delta | \lambda)
$$
where
\begin{align*}
    {\cal A}(\delta | \lambda)
    = & \sum_{x\in {\cal X}} \sum_{y\in {\cal Y}} \tilde{p}(x,y) \sum_i \delta_i f_i(x,y) + 1 - \sum_{x\in {\cal X}} \tilde{p}(x) \frac{Z_{\lambda + \delta}(x)}{Z_{\lambda}(x)} \\
    = & \sum_{x\in {\cal X}} \sum_{y\in {\cal Y}} \tilde{p}(x,y) \sum_i \delta_i f_i(x,y) + 1 - \sum_{x\in {\cal X}} \tilde{p}(x) \sum_{y\in {\cal Y}} p_{\lambda}(y|x) \exp \left( \sum_i \delta_i f_i(x,y) \right)
\end{align*}
If we find $\delta$ for which \ ${\cal A}(\delta | \lambda) > 0$, then $\lambda + \delta$ is an improvement over $\lambda$.

**problem:**
the straightforward maximisation of ${\cal A}(\delta | \lambda)$ may not work, because differentiating ${\cal A}(\delta | \lambda)$ with respect to $\delta_i$ yields an equation containing $\{ \delta_1, \ldots, \delta_n \}$ (ie, $\delta_i$ is coupled).

**solution:**
to get around above problem, we introduce the quantity: $\displaystyle f^{\#}(x,y) \equiv \sum_i f_i(x,y)$, ie, the number of features that are not zero at $(x,y)$, and rewrite:
$$
    \exp \left( \sum_i \delta_i f_i(x,y) \right) = \exp \left( \sum_i \frac{f_i(x,y)}{f^{\#}(x,y)} \delta_i f^{\#}(x,y) \right)
$$

Notice that $\displaystyle \frac{f_i(x,y)}{f^{\#}(x,y)}$ is a pdf over $i$, that is non-negative and sums to one.

For a convex function, $f(x) = e^x$, *Jensen's inequality* says
\begin{align*}
    f(E[i]) &\leq E[f(i)] \\
    \exp \left( \sum_i p(i) q(i) \right) &\leq \sum_i p(i) \exp q(i) \\
    \exp \left( \sum_i \frac{f_i(x,y)}{f^{\#}(x,y)} \delta_i f^{\#}(x,y)\right) &\leq \sum_i \frac{f_i(x,y)}{f^{\#}(x,y)} \exp \left( \delta_i f^{\#}(x,y) \right)
\end{align*}
which results in ${\cal A}(\delta | \lambda) \geq {\cal B}(\delta | \lambda)$ where
$$
    {\cal B}(\delta | \lambda) = \sum_{x\in {\cal X}} \sum_{y\in {\cal Y}} \tilde{p}(x,y) \sum_i \delta_i f_i(x,y) + 1 - \sum_{x\in {\cal X}} \tilde{p}(x) \sum_{y\in {\cal Y}} p_{\lambda}(y|x) \sum_i \frac{f_i(x,y)}{f^{\#}(x,y)} \exp \left( \delta_i f^{\#}(x,y) \right)
$$

${\cal B}(\delta | \lambda)$ is a new, not as tight, lower bound on the change in log likelihood:
$$
    {\cal L}(\tilde{p} | p_{\lambda + \delta}) - {\cal L}(\tilde{p} | p_{\lambda}) \geq {\cal B}(\delta | \lambda)
$$

Differentiating ${\cal B}(\delta | \lambda)$ with respect to $\delta_i$,
$$
    \frac{\partial {\cal B}(\delta | \lambda)}{\partial \delta_i} = \overbrace{\sum_{x\in {\cal X}} \sum_{y\in {\cal Y}} \tilde{p}(x,y) f_i(x,y)}^{\tilde{p}(f_i)} - \sum_{x\in {\cal X}} \tilde{p}(x) \sum_{y\in {\cal Y}} p_{\lambda}(y|x) f_i(x,y) \exp \left( \delta_i f^{\#}(x,y) \right)
$$

What is nice about this? $\quad\Longrightarrow\quad$ $\delta_i$ appears alone without any coupling.

**Iterative scaling algorithm (ISA)**

1. initially set $\lambda_i = 0$ for all $i = 1,\ldots,n$

2. do for each $i = 1,\ldots,n$ :

    A. let $\delta_i$ be the solution to
    $$
        \displaystyle \sum_{x\in {\cal X}} \tilde{p}(x) \sum_{y\in {\cal Y}} p_{\lambda}(y|x) f_i(x,y) \exp\left( \delta_i f^{\#}(x,y) \right) = \tilde{p}(f_i)
    $$
    where $\displaystyle f^{\#}(x,y) \equiv \sum_{i = 1,\ldots,n} f_i(x,y)$

	B. update multipliers by $\lambda_i \leftarrow \lambda_i + \delta_i$

3. go back to step 2, if not all $\lambda_i$ have converged

**British weather forecast problem (revisited)**

The numbers of active features for \{misty, foggy, cloudy, sunny, rainy\} are 3, 2, 2, 1, and 1, respectively.

We denote $a = e^{\delta_1}$, $b = e^{\delta_2}$ and $c = e^{\delta_3}$, and define:
\begin{align*}
    {\cal G}(a) &= p(misty)\cdot a^3 + p(foggy)\cdot a^2 + p(cloudy)\cdot a^2 + p(sunny)\cdot a + p(rainy)\cdot a - 1 \\
	{\cal G}(b) &= p(misty)\cdot b^3 + p(foggy)\cdot b^2 - 0.3 \\
	{\cal G}(c) &= p(misty)\cdot c^3 + p(cloudy)\cdot c^2 - 0.5
\end{align*}

To get $a$, $b$, and $c$, we achieve the *Newton-Raphson*, ie, $\displaystyle a_{n+1} = a_n - \frac{{\cal G} (a_n)}{{\cal G}' (a_n)}$ .

We are able to calculate probabilities from multipliers $\lambda_1$, $\lambda_2$, and $\lambda_3$ :
\begin{align*}
    p(misty) &= \frac{1}{Z_{\lambda}} \exp (\lambda_1 + \lambda_2 + \lambda_3) \\
	p(foggy) &= \frac{1}{Z_{\lambda}} \exp (\lambda_1 + \lambda_2) \\
	p(cloudy) &= \frac{1}{Z_{\lambda}} \exp (\lambda_1 + \lambda_3) \\
    p(sunny) &= \frac{1}{Z_{\lambda}} \exp \lambda_1 \\
	p(rainy) &= \frac{1}{Z_{\lambda}} \exp \lambda_1
\end{align*}

Here's the graph, produced by applying the *iterative scaling algorithm*:
<img src="./figs/newton.jpg", width=500, align=center>

Comparison of numerical calculation (up to 20 iterations) and analytical calculation:
  
.             |  0    | 1     | $\cdots$ | 20    | | analytical
--------------|-------|-------|----------|-------|-|-----------
$p($misty$)$  | 0.333 | 0.168 |          | 0.186 | | 0.186
$p($foggy$)$  | 0.222 | 0.175 |          | 0.115 | | 0.114
$p($cloudy$)$ | 0.222 | 0.213 |          | 0.312 | | 0.314
$p($sunny$)$  | 0.111 | 0.222 |          | 0.193 | | 0.193
$p($rainy$)$  | 0.111 | 0.222 |          | 0.193 | | 0.193
